In [1]:
import os
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import re

In [2]:
# Ruta a la carpeta donde están los archivos
carpeta_archivos = 'GOES_Files/ACMF/'
output_folder = 'Images/ACMF/'

# Crear la carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

In [3]:
# Rango de fechas
#fecha_inicio = '2023-07-01 05:00:00'
#fecha_fin = '2023-07-07 18:30:00'

fecha_inicio = datetime.strptime('2023-07-11 05:00:00', '%Y-%m-%d %H:%M:%S')
fecha_fin = datetime.strptime('2023-07-12 18:30:00', '%Y-%m-%d %H:%M:%S')

In [4]:
ncs = [f'GOES_Files/ACMF/{i}' for i in os.listdir(carpeta_archivos) if i.endswith('.nc')]
ncs = sorted(ncs)

In [5]:
# Lista para almacenar los datos
das = []

for nc in ncs:
    # Extraer la fecha desde el nombre del archivo usando regex
    match = re.search(r's(\d{4})(\d{2})(\d{2})(\d{2})(\d{2})(\d{2})', nc)
    if match:
        # Convertir la fecha extraída al formato datetime
        fecha_archivo = datetime(
            year=int(match.group(1)), 
            month=int(match.group(2)), 
            day=int(match.group(3)),
            hour=int(match.group(4)), 
            minute=int(match.group(5)), 
            second=int(match.group(6))
        )
        
        # Filtrar archivos según el rango de fechas
        if fecha_inicio <= fecha_archivo <= fecha_fin:
            ds = xr.open_dataset(nc).sel(x=slice(-0.01, 0.03), y=slice(0.05, 0.01))
            
            # Añadir el dataset a la lista para concatenar
            das.append(ds)

In [6]:
# Concatenar todos los datasets y estandarizar el tiempo a intervalos de 10 minutos
ds_col = xr.concat(das, dim='t')
ds_col['t'] = [pd.Timestamp(i, tz='UTC').floor('10min') for i in ds_col.t.values]

In [7]:
# Guardar imágenes para cada archivo procesado
for i, ds in enumerate(ds_col.groupby('t')):
    tiempo = ds[0]
    variable = ds[1]['ACM']  # Usar la variable 'ACM'
    variable.plot()

    # Convertir el valor de tiempo a una cadena adecuada para un nombre de archivo
    tiempo_str = tiempo.strftime('%Y%m%d_%H%M%S')

    # Guardar la figura en el folder especificado como JPG, usando el valor de 't' como nombre
    output_path = os.path.join(output_folder, f"ACMF_{tiempo_str}.jpg")
    plt.savefig(output_path, format='jpg')
    
    # Cerrar la figura para liberar memoria
    plt.close()